### Part2. 计算无常损失

无常损失（Impermanent Loss）的定义：

$$ IL = \frac{Loss from LP}{hodl} = \frac { v_{1} - v_{hodl} }{ v_{hodl} } $$

其中v1是当前流动性头寸中两种token价值的总和。

1. 找一个合适的价格api

2. 获取当前流动性包含多少token1，token2的数据

3. 计算出当前头寸的价值，从而计算出无常损失


In [33]:
from datetime import datetime, timedelta
from importlib import reload
from poolData import swapData
reload(swapData)

query = swapData.SwapDataQuery()

#### （一）获得某一时间下头寸内两种token的数量组成

首先需要查询仓位在起始时间的token数量和在结束时间的token组成，可以用snapshot获取

positionSnapshots中通过id来查询的，id由两个部分组成，例如"id": "100000#12926073"，表示position的唯一标识符id是100000，该snapshot的区块高度是12926073。

现在已知初始创建头寸的时候的区块高度为16859930，现在的区块高度大概是16953257。

所以需要查询的id是469767#16859930和469767#16953257。

初始的组成可以直接获取到，但是当前时间下的组成可能需要计算一下（合约中不会直接记录这个数据）

当前头寸两种token的数量组成是随着当前价格的变化而变化的，可以用以下的公式推导出来：

已知p_{current}和L，如何求x和y？
$$
\begin{cases}
 x \cdot y = L ^ 2  \\
 P_{c} = \frac{y}{x}
\end{cases}
$$

整理得到：

$$
\begin{cases}
 x \frac{L}{\sqrt{p_{current}}}  \\
 y = L \cdot \sqrt{P_{current}}
\end{cases}
$$

但是要注意这里的x和y是加上了virtual liquidity之后的，如果要获取用户实际上拥有的x和ytoken是多少，还需要减去$x_{v}$和$y_{v}$。$x_{v}$和$y_{v}$的公式如下：
$$
\begin{cases}
 x_{virtual} = \frac{L}{\sqrt{p_{upper}}}  \\
 y_{virtual} = L \cdot \sqrt{P_{lower}}
\end{cases}
$$

不过在计算il的时候用不着求virtual liquidity，因为算delta的时候抵消了。

$$ IL = \frac { v_{1} - v_{hodl} }{ v_{hodl} } = \frac { (x_{1} \cdot P_{x} + y_{1} \cdot P_{y}) - (x_{0} \cdot P_{x} + y_{0} \cdot P_{y})}{ x_{0} \cdot P_{x} + y_{0} \cdot P_{y} } $$

整理得到：

$$ IL = \frac { (x_{1} - x_{0}) \cdot P_{x} + (y_{1} - y_{0}) \cdot P_{y} }{ x_{0} \cdot P_{x} + y_{0} \cdot P_{y} } = \frac { (x_{1} - x_{0}) \cdot P_{x} + (y_{1} - y_{0}) \cdot P_{y} }{ x_{0} \cdot P_{x} + y_{0} \cdot P_{y} } = \frac{\Delta x \cdot P_current + \Delta y \cdot P_current }{x_{0} \cdot P_current + y_{0} \cdot P_current} $$



In [49]:
position_id = "466521"
begin_block = "16802135"
end_block = "17049590"

begin_position_data = query.query_position_snaps(position_id=position_id, begin_block=begin_block)
end_position_data = query.query_position_snaps(position_id=position_id, begin_block=begin_block, current_block=end_block)

In [50]:
end_position_data

""


In [51]:
begin_position_data

""


### 参数解释

1. depositedToken0和position.depositedToken1

    depositedToken0的数据和[初始化头寸的交易](https://etherscan.io/tx/0xe2e2041d2b09d5d7ae4f999951775f7ad527a64f3ad103584bfdb34307056a9d)中显示的数据是一样的，所以depositedToken0的数据表示首次创建流动性头寸的时候存入的token。position.depositedToken1的数量要更大，表示在这一段区间内，lper应该是又给池子补充了流动性。

    为了简化，我们先不考虑后期调整头寸（补充/取出）的操作，所以暂时我们先拿最初始的来计算（这样算出来手续费会偏多，因为用的l是补充之后的l）

2. liquidity

    在v3中计算liquidity会比v2复杂一些，大致流程如下：
    ![liquidity-calculations](formulas/liquidity-calculations.jpg)

    另外，这里获取到的liquidity是以

In [ ]:
# 好啦现在可以开始计算当前价格下头寸的token数量了 以下都用x表示token0，y表示token1, t0表示初始时间，t1表示现在的时间
# 首先初始时间的可以直接拿到：
x_t0= float(begin_position_data["depositedToken0"][0])
y_t0= float(begin_position_data["depositedToken1"][0])

# 然后用上面的公式来算当前的：
# 获取价格
end = datetime.now()
begin = end - timedelta(hours=1)
pool_id = "0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640"
price_data = query.query_liquidity(begin=int(begin.timestamp()), end=int(end.timestamp()), pool_id=pool_id)
price_data["sqrtPrice"] = price_data["sqrtPrice"].astype(float)
price_current = ((price_data["sqrtPrice"][0] / (2**96) ) ** 2 )

# 获取L
position_liquidity = float(begin_position_data["liquidity"][0])

x_t1= (position_liquidity / (price_current ** 0.5) ) * (10 ** -6)
y_t1= (position_liquidity * (price_current ** 0.5)) * (10 ** -18)

# 数量是对的，但是怎么小数点后了一位🥲，我一会看看为什么 先除10先
x_t1 = x_t1/10
y_t1 = y_t1/10
print("Initialized amount of x token: ", x_t0)
print("Initialized amount of y token: ", y_t0)
print("Current amount of x token: ", x_t1)
print("Current amount of y token: ", y_t1)

In [ ]:
begin_position_data["depositedToken1"][0]
begin_position_data["depositedToken0"][0]

In [ ]:
price_current / (10 ** 12)

In [ ]:
x_t1

In [ ]:
(position_liquidity / (price_current ** 0.5) ) * (10 ** -6)

In [ ]:
(position_liquidity * (price_current ** 0.5)) * (10 ** -18)

In [ ]:
# 诶不对啊 头寸里不管如何变化，应该是一种币增加另一种减少才对，这里怎么两种都一起减少了？
# 感觉问题应该出在liquidity上
# liquidity

#### （二）计算IL
计算IL的基准单位一般选择为美元，所以还需要一个价格api，不过我这里选择的币对比较特殊，token0是usdc，所以直接用token0来计算。

In [ ]:
price_y = (1 / ((price_data["sqrtPrice"][0] / (2**96) ) ** 2 )) * (10 ** 12)

In [ ]:
delta_x = x_t1 - x_t0
delta_y = y_t1 - y_t0
IL_neat = delta_x + price_y * delta_y
print("The neat Impermanent Loss is", IL_neat)